In [138]:
import pandas as pd
import nltk
import re
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords


In [139]:
file_path ='../Online_Retail_classification_MLmodel/Retail_data_combined.csv'
df= pd.read_csv(file_path)
df.head()

,Unnamed: 0,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country
0,0,489434,85048,15CM CHRISTMAS GLASS BALL 20 LIGHTS,12,12/1/2009 7:45,6.95,13085.0,United Kingdom
1,1,489434,79323P,PINK CHERRY LIGHTS,12,12/1/2009 7:45,6.75,13085.0,United Kingdom
2,2,489434,79323W,WHITE CHERRY LIGHTS,12,12/1/2009 7:45,6.75,13085.0,United Kingdom
3,3,489434,22041,"RECORD FRAME 7"" SINGLE SIZE",48,12/1/2009 7:45,2.10,13085.0,United Kingdom
4,4,489434,21232,STRAWBERRY CERAMIC TRINKET BOX,24,12/1/2009 7:45,1.25,13085.0,United Kingdom


# Tokenization and removing stop words

In [140]:
#pip install nltk

In [141]:
# Initialize NLTK tokenizer
nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\gothw\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\gothw\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [142]:
data = df['Description']

In [143]:
df_1=pd.DataFrame(data)
print(df_1)

                                 Description
0        15CM CHRISTMAS GLASS BALL 20 LIGHTS
1                         PINK CHERRY LIGHTS
2                        WHITE CHERRY LIGHTS
3               RECORD FRAME 7" SINGLE SIZE 
4             STRAWBERRY CERAMIC TRINKET BOX
...                                      ...
1067366         CHILDREN'S APRON DOLLY GIRL 
1067367        CHILDRENS CUTLERY DOLLY GIRL 
1067368      CHILDRENS CUTLERY CIRCUS PARADE
1067369        BAKING SET 9 PIECE RETROSPOT 
1067370                              POSTAGE

[1067371 rows x 1 columns]


In [144]:
df_1.isna().sum()

Description    4382
dtype: int64

In [145]:
df_1 = df_1.dropna()

In [146]:
df_1.isna().sum()

Description    0
dtype: int64

In [147]:
# create function to clean text
def text_cleaner(text):
    text = re.sub(r'\d+', '', text) # remove numerical digits
    text = re.sub(r'[^\w\s]', '', text) # remove punctuation
    text = re.sub(r'\s+', ' ', text) # remove white space
    text = text.lower() # make all text lowercase
    return text

In [148]:
# apply text_cleaner function to data
df_1['Description'] = df_1['Description'].apply(text_cleaner)

In [149]:
df_1.head()

,Description
0,cm christmas glass ball lights
1,pink cherry lights
2,white cherry lights
3,record frame single size
4,strawberry ceramic trinket box


In [150]:
df_1['tokenized_column'] = df_1['Description'].apply(word_tokenize)
df_1.head()

,Description,tokenized_column
0,cm christmas glass ball lights,"[cm, christmas, glass, ball, lights]"
1,pink cherry lights,"[pink, cherry, lights]"
2,white cherry lights,"[white, cherry, lights]"
3,record frame single size,"[record, frame, single, size]"
4,strawberry ceramic trinket box,"[strawberry, ceramic, trinket, box]"


In [151]:
stop_words = set(stopwords.words('english'))

In [152]:
def remove_stopwords(tokens):
    return [word for word in tokens if word.lower() not in stop_words]

In [153]:
df_1['tokenized_column'] = df_1['tokenized_column'].apply(remove_stopwords)

In [154]:
# Adding a coulmn with the count of the 
df_1['word_count'] = df_1['tokenized_column'].apply(len)


In [155]:
df_1.head()

,Description,tokenized_column,word_count
0,cm christmas glass ball lights,"[cm, christmas, glass, ball, lights]",5
1,pink cherry lights,"[pink, cherry, lights]",3
2,white cherry lights,"[white, cherry, lights]",3
3,record frame single size,"[record, frame, single, size]",4
4,strawberry ceramic trinket box,"[strawberry, ceramic, trinket, box]",4


In [162]:
from collections import Counter

# Sample data with tokenized column
data = df_1['tokenized_column']

df_2 = pd.DataFrame(data)
df_2.head()


,tokenized_column
0,"[cm, christmas, glass, ball, lights]"
1,"[pink, cherry, lights]"
2,"[white, cherry, lights]"
3,"[record, frame, single, size]"
4,"[strawberry, ceramic, trinket, box]"


In [163]:
# Combine tokenized words into a single list
all_words = [word for words in df_2['tokenized_column'] for word in words]

# Calculate word frequencies using Counter
word_frequencies = Counter(all_words)

# Convert word frequencies to a DataFrame
word_frequency_df = pd.DataFrame(word_frequencies.items(), columns=['Word', 'Frequency'])

# Sort DataFrame by frequency in descending order
word_frequency_df = word_frequency_df.sort_values(by='Frequency', ascending=False)

print(word_frequency_df)

           Word  Frequency
67          set     115058
142         bag      92682
150         red      90500
31        heart      78666
5          pink      63542
...         ...        ...
2417       asos          1
2422      sizes          1
2423   delivery          1
1988    returns          1
2431  awayrusty          1

[2640 rows x 2 columns]


# Term Frequency-Inverse Document Frequency Weight

In [156]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [157]:
#hashing = HashingTF(inputCol ='tokenized_column', outputCol='hashedValues', numFeatures=pow(2,18))

#hashed_df = hashing.transform(removed_frame)
#hashed_df.show(truncate=False)

In [158]:
df_1['combined_text'] = df_1['tokenized_column'].apply(lambda tokens: ' '.join(tokens))

In [159]:
vectorizer = TfidfVectorizer()

In [160]:
tfidf_matrix = vectorizer.fit_transform(df_1['combined_text'])

In [161]:
tfidf_df = pd.DataFrame(tfidf_matrix.toarray(), columns=vectorizer.get_feature_names_out())

MemoryError: Unable to allocate 20.8 GiB for an array with shape (1062989, 2623) and data type float64